# Imports

In [69]:
#!pip install -U scikit-learn

In [70]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean

# Input

In [71]:
transformation = Transformation()

user_number_triangles = 120
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [72]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [73]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [74]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [75]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[0.12922126 0.12929645 0.12922126 ... 0.12030065 0.17462619 0.12030065]
[0.41136764 0.41160697 0.41136764 ... 0.38296942 0.55591131 0.38296942]
[0.63338925 0.63346312 0.63338925 ... 0.62458092 0.67675747 0.62458092]
(5999,)


### Multinomial Sampling

In [76]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[1 0 0 ... 0 0 0]


In [77]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[92.37709   , -0.33300498,  1.0477394 ],
       [77.4143    , -6.5098743 , 82.76787   ],
       [77.03556   , -6.5098743 , 83.235565  ],
       [78.179504  , -7.6759295 , 84.37951   ],
       [86.072075  , -0.33300498, -9.87287   ]], dtype=float32)

# KNN extended graph

In [78]:
_, indices = NearestNeighbors(n_neighbors=10).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 360
Number of edges: 1934


# Edge Predictor

In [79]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(360, 64)

In [80]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [81]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

# Face Candidates

#### Inputs

In [82]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 140)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 75)	1
  (1, 140)	1
  (2, 0)	1
  (2, 1)	1
  (2, 3)	1
  (2, 4)	1
  (2, 5)	1
  :	:
  (357, 331)	1
  (357, 335)	1
  (357, 339)	1
  (357, 350)	1
  (357, 355)	1
  (357, 356)	1
  (357, 358)	1
  (358, 261)	1
  (358, 346)	1
  (358, 347)	1
  (358, 348)	1
  (358, 349)	1
  (358, 350)	1
  (358, 351)	1
  (358, 356)	1
  (358, 357)	1
  (359, 135)	1
  (359, 165)	1
  (359, 166)	1
  (359, 192)	1
  (359, 271)	1
  (359, 303)	1
  (359, 304)	1
  (359, 318)	1
  (359, 337)	1
[[8.70408930e-02 8.44462104e-02 8.34451601e-02 ... 6.39117555e-02
  6.39117562e-02 2.94004247e-02]
 [5.09324629e+00 1.16337407e-01 1.17325638e-01 ... 1.34968487e-01
  1.34968486e-01 1.53294940e-01]
 [4.28683024e+00 1.01425741e+00 1.06441104e-01 ... 1.50512441e-01
  1.50512439e-01 2.80146827e-01]
 ...
 [5.51395843e+01 5.18114756e+01 6.048

In [83]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0

A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T

A_s = A_s/A_s.max()                                         # Normalize

In [84]:
print(A_s)  # symmétrique
print(A_s.shape)

[[1.08905957e-41 1.60169161e-41 1.47178456e-41 ... 4.81206599e-33
  2.06337574e-33 5.18288388e-22]
 [1.60169161e-41 2.41371931e-41 2.44109696e-41 ... 8.66937426e-33
  4.03617927e-33 9.76324729e-22]
 [1.47178456e-41 2.44109696e-41 2.46819773e-41 ... 1.07907859e-32
  5.24019620e-33 1.22710974e-21]
 ...
 [4.81206599e-33 8.66937426e-33 1.07907859e-32 ... 3.29914247e-23
  1.71850582e-23 4.01335097e-12]
 [2.06337574e-33 4.03617927e-33 5.24019620e-33 ... 1.71850582e-23
  1.23751800e-23 3.19616367e-12]
 [5.18288388e-22 9.76324729e-22 1.22710974e-21 ... 4.01335097e-12
  3.19616367e-12 6.25652995e-01]]
(360, 360)


# Face Classifier

### TriConv

#### Inputs

In [85]:
triangles = list(nx.simple_cycles(extended_graph, length_bound=3))  # [triangle0, triangle1,...] | triangle0 = [node1,node2,node3] | node1 = (x ,y ,z)
# triangles = cycles
print(triangles[0])
print(np.array(triangles).shape)    #nb_triangle, 3 nodes, 3 dimensions par node

[(39.0, 8.793697, 89.66699), (51.0, 8.642877, 91.6555), (39.0, 4.590885, 82.49242)]
(3912, 3, 3)


In [86]:
p_init = np.zeros((len(triangles)))

for index_triangle, triangle in enumerate(triangles):
    i = list(dict(extended_graph._node).keys()).index(triangle[0])
    j = list(dict(extended_graph._node).keys()).index(triangle[1])
    k = list(dict(extended_graph._node).keys()).index(triangle[2])
    p_init[index_triangle] = (A_s[i,j] + A_s[i,k] + A_s[j,k])/3
print(p_init)
print(len(p_init))

[4.58781717e-19 4.30254077e-19 4.93753535e-19 ... 4.92285582e-24
 3.31698803e-24 2.85891712e-23]
3912


#### Calculate barycenter

In [87]:
barycenters = list()

for _, triangle in enumerate(triangles):
    b_x = (triangle[0][0] + triangle[1][0] + triangle[2][0]) / 3
    b_y = (triangle[0][1] + triangle[1][1] + triangle[2][1]) / 3
    b_z = (triangle[0][2] + triangle[1][2] + triangle[2][2]) / 3
    barycenters.append([b_x, b_y, b_z])

print(barycenters[0])
print(len(barycenters))

[43.0, 7.342486063639323, 87.9383036295573]
3912


#### KNN Tri

In [88]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)


Diff Barycenters

In [89]:
barycenters_array = np.array(barycenters)
indices_neigh_tri_array = np.array(indices_neigh_tri)

# Calculate differences between points
barycenters_diff = np.subtract(barycenters_array[indices_neigh_tri_array[:, 0]][:, np.newaxis], barycenters_array[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(3912, 19, 3)


#### calculate e norm matrix

In [90]:
diff_vectors = list()

for _, triangle in enumerate(triangles):
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])
print(diff_vectors)
print(len(diff_vectors))

[[12.164577, 8.314937, 15.632683], [12.164577, 8.895975, 15.984401], [12.164577, 20.183048, 16.895582], [12.164577, 21.091703, 17.571638], [12.164577, 11.594044, 17.706322], [12.164577, 14.27775, 19.485369], [12.164577, 17.220001, 21.236633], [17.521263, 8.314937, 14.656956], [17.521263, 8.895975, 14.277754], [17.521263, 20.183048, 13.097498], [17.521263, 21.091703, 12.073596], [17.521263, 11.594044, 12.08682], [17.521263, 14.27775, 8.895974], [17.521263, 17.220001, 1.9942106], [17.220001, 8.314937, 13.458566], [17.220001, 8.895975, 13.01975], [17.220001, 20.183048, 12.448402], [17.220001, 21.091703, 12.018468], [17.220001, 11.594044, 10.559403], [17.220001, 14.27775, 7.1179705], [14.27775, 8.314937, 7.7219834], [14.27775, 8.895975, 7.1179705], [14.27775, 20.183048, 12.637379], [14.27775, 21.091703, 14.269865], [14.27775, 11.594044, 3.9627445], [11.594044, 8.314937, 3.9627426], [11.594044, 8.895975, 3.3109968], [11.594044, 20.183048, 14.269864], [11.594044, 21.091703, 16.336151], [8.89

#### Calculate r

In [91]:
max_diff_vectors = np.array(diff_vectors).max(axis=1)       # calculate t_n_max
min_diff_vectors = np.array(diff_vectors).min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(3912,)

In [92]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri_array[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri_array[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri_array[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(3912, 19)
(3912, 19)


In [93]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))


# Fill r_matrix with the corresponding indices from indices_neigh_tri
r_matrix[:, :, 0] = min_diff_vectors_diff
r_matrix[:, :, 1] = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(3912, 19, 5)


#### Calculate f

In [94]:
# MLP * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999999758
(3912,)
[0.00025523 0.00025523 0.00025523 ... 0.00025523 0.00025523 0.00025523]


C:\Users\ahdel\AppData\Local\Temp\ipykernel_11880\1064855114.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))


# Simplified Mesh

In [95]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[:1])

(120, 3, 3)
[[[78.490616  -8.610001  87.24593  ]
  [78.179504  -7.6759295 84.37951  ]
  [79.10347   -8.610001  86.84794  ]]]


In [96]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 71
Number of edges: 174


In [97]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)